In [1]:
import warnings
warnings.filterwarnings('ignore')

import geopandas as gpd
from shapely import geometry
import pandas as pd

In [36]:
FILE_NAME = "VA0Precinct.json"

In [37]:
gdf = gpd.read_file(FILE_NAME)

DriverError: VA0Precinct.json: No such file or directory

In [9]:
gdfcopy = gdf.copy()
gdf.head(2)

,County,Precinct #,Hillary Clinton/Dem,Donald J. Trump/Rep,Gary Johnson/Lib,Jill Stein/Grn,Other/Other,Total Votes,State,CName,Neighbors,Total Population,White,Black or African American,American Indian,Asian,Other Population,geometry
0,Bristol,0101,1027,522,60,23,2,1634,ri,ri-bristol-0101,"ri-bristol-0101,ri-bristol-0102; ri-bristol-01...",3268,3055,23,3,110,73,"POLYGON ((-71.32270 41.77270, -71.32270 41.772..."
1,Bristol,0102,1331,683,68,21,3,2106,ri,ri-bristol-0102,"ri-bristol-0102,ri-bristol-0101; ri-bristol-01...",2921,2740,18,3,85,70,"MULTIPOLYGON (((-71.35581 41.74533, -71.35560 ..."


# create x y for each polygon

In [4]:
Polygon = geometry.polygon.Polygon
MultiPolygon = geometry.multipolygon.MultiPolygon

In [9]:
# create a new df
# columns = ["index", "shape_index", "polygon_index", "Precinct #", "County", "Precinct Name", "Hillary Clinton/Dem",
#            "Donald J. Trump/Rep", "Gary Johnson/Lib", "Jill Stein/Grn", "Other/Other",
#           "Total Votes", "X_Coord", "Y_Coord"]

In [15]:
def parse(gdf):
    columns = ["geovertex_id", "shape_id", "polygon_id", "x_pos", "y_pos"]
    df = pd.DataFrame(columns=columns)

    counter = 0
    i = 1
    shape_id= 1
    polygon_id = 1

    for index, row in gdf.iterrows():
        # for visualization only
        if counter%10 == 0:
            break
            print(" {:.2f}% ".format(counter/len(gdf)*100), end="")
        else:
            print("#", end="")

        cur_shape = row["geometry"]
        cur_type = type(cur_shape)

        row = row.drop(["geometry"])
        cur_row = row
        cur_row["shape_id"] = shape_id
        shape_id += 1

        if cur_type == MultiPolygon:
            for polygon in cur_shape:
                cur_row["polygon_id"] = polygon_id
                polygon_id += 1
                for coords in polygon.exterior.coords:
                    cur_row["geovertex_id"] = i
                    i += 1
                    cur_row["x_pos"] = coords[0]
                    cur_row["y_pos"] = coords[1]
                    df = df.append(cur_row, ignore_index=True)
                    cur_row = row
        elif cur_type == Polygon:
            cur_row["polygon_id"] = polygon_id
            polygon_id += 1
            for coords in cur_shape.exterior.coords:
                cur_row["geovertex_id"] = i
                i += 1
                cur_row["x_pos"] = coords[0]
                cur_row["y_pos"] = coords[1]
                df = df.append(cur_row, ignore_index=True)
                cur_row = row
        counter += 1
        
    return df

In [4]:
# drop some columns
dropping = gdf.columns.to_list()
dropping.remove("geometry")

gdf.drop(dropping, inplace=True, axis=1)
gdf.head(1)

,geometry
9501,"POLYGON ((-75.66233 37.77053, -75.66244 37.770..."


In [24]:
gdf_Parsed = parse(gdf)

##########

In [26]:
name = FILE_NAME.split(".")[0]
gdf_Parsed.to_csv(name+"_Parsed.csv", index=False)